In [1]:
from github import Auth, Github

from getpass import getpass

token = getpass("请输入 GitHub Token: ")
auth = Auth.Token(token)

g = Github(auth=auth)

ORG_NAME = "any-xi"
CONTAINER_NAME_PREFIX = "writing-"


AssertionError: 

In [2]:
from typing import TypedDict

# 获取 Docker package
# 获取组织
org = g.get_organization(ORG_NAME)


class Package(TypedDict):
    id: int
    name: str
    version_count: int


packages: list[Package] = []

# 获取组织下的所有 packages
# 注意: PyGithub 目前不支持直接通过 Organization 对象获取 packages
# 需要使用 REST API 直接调用
headers, packages = g._Github__requester.requestJsonAndCheck(
    "GET",
    f"/orgs/{ORG_NAME}/packages",
    {"package_type": "container"},  # 指定 package_type 为 container
)


# 遍历并打印 packages 信息
packages = [pkg for pkg in packages if pkg["name"].startswith(CONTAINER_NAME_PREFIX)]

print([p["name"] for p in packages])

['writing-frontend', 'writing-backend', 'writing-streamlit_app', 'writing-reader', 'writing-searxng']


In [3]:
# 解析创建时间并计算是否超过一个月
from datetime import datetime, timezone, timedelta


def get_outdated_versions(pkg_name: str):
    headers, versions = g._Github__requester.requestJsonAndCheck(
        "GET", f"/orgs/{ORG_NAME}/packages/container/{pkg_name}/versions"
    )

    outdated_versions = []

    # 遍历并打印版本信息
    for version in versions:
        created_at = datetime.strptime(version["created_at"], "%Y-%m-%dT%H:%M:%SZ")
        created_at = created_at.replace(tzinfo=timezone.utc)
        one_month_ago = datetime.now(timezone.utc) - timedelta(days=30)

        tags = version["metadata"]["container"]["tags"]

        if created_at < one_month_ago and not tags:
            outdated_versions.append(version)

    return outdated_versions


def delete_package(pkg_name: str, version_id: int):
    print(f"Deleting {pkg_name} version {version_id}")
    g._Github__requester.requestJsonAndCheck(
        "DELETE",
        f"/orgs/{ORG_NAME}/packages/container/{pkg_name}/versions/{version_id}",
    )


for pkg in packages:
    outdated_versions = get_outdated_versions(pkg["name"])

    for version in outdated_versions:
        delete_package(pkg["name"], version["id"])


Deleting writing-frontend version 245389973
Deleting writing-frontend version 245340222
Deleting writing-frontend version 245229660
Deleting writing-frontend version 244899142
Deleting writing-frontend version 244153705
Deleting writing-frontend version 244075000
Deleting writing-frontend version 243731620
Deleting writing-frontend version 242966017
Deleting writing-frontend version 242942322
Deleting writing-frontend version 242748272
Deleting writing-frontend version 242362897
Deleting writing-frontend version 242194402
Deleting writing-frontend version 242188201
Deleting writing-frontend version 242109434
Deleting writing-backend version 245389600
Deleting writing-backend version 245385936
Deleting writing-backend version 245339618
Deleting writing-backend version 245229725
Deleting writing-backend version 244899585
